## Classification
Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

In [1]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [2]:
using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

Get the data first

In [3]:
iris = dataset("datasets", "iris")

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa
7,4.6,3.4,1.4,0.3,setosa
8,5.0,3.4,1.5,0.2,setosa
9,4.4,2.9,1.4,0.2,setosa


In [4]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

150-element CategoricalArrays.CategoricalArray{String,1,UInt8}:
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 "setosa"
 ⋮
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"
 "virginica"

In [5]:
X

150×4 Matrix{Float64}:
 5.1  3.5  1.4  0.2
 4.9  3.0  1.4  0.2
 4.7  3.2  1.3  0.2
 4.6  3.1  1.5  0.2
 5.0  3.6  1.4  0.2
 5.4  3.9  1.7  0.4
 4.6  3.4  1.4  0.3
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.9  3.1  1.5  0.1
 5.4  3.7  1.5  0.2
 4.8  3.4  1.6  0.2
 4.8  3.0  1.4  0.1
 ⋮              
 6.0  3.0  4.8  1.8
 6.9  3.1  5.4  2.1
 6.7  3.1  5.6  2.4
 6.9  3.1  5.1  2.3
 5.8  2.7  5.1  1.9
 6.8  3.2  5.9  2.3
 6.7  3.3  5.7  2.5
 6.7  3.0  5.2  2.3
 6.3  2.5  5.0  1.9
 6.5  3.0  5.2  2.0
 6.2  3.4  5.4  2.3
 5.9  3.0  5.1  1.8

In [6]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [7]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [8]:
?randsubseq

search: randsubseq randsubseq! RandomSub StratifiedRandomSub



```
randsubseq([rng=GLOBAL_RNG,] A, p) -> Vector
```

Return a vector consisting of a random subsequence of the given array `A`, where each element of `A` is included (in order) with independent probability `p`. (Complexity is linear in `p*length(A)`, so this function is efficient even if `p` is small and `A` is large.) Technically, this process is known as "Bernoulli sampling" of `A`.

# Examples

```jldoctest
julia> rng = MersenneTwister(1234);

julia> randsubseq(rng, 1:8, 0.3)
2-element Vector{Int64}:
 7
 8
```


In [9]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

46-element Vector{Int64}:
   2
   8
   9
  12
  14
  17
  20
  25
  28
  29
  36
  38
  39
   ⋮
  98
 105
 107
 110
 111
 113
 122
 128
 129
 133
 144
 146

We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous.

In [10]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assign_class (generic function with 1 method)

### 🟣 Method 1: Lasso

In [11]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

Least Squares GLMNet Cross Validation
72 models for 4 predictors in 10 folds
Best λ 0.001 (mean loss 0.046, std 0.007)

In [12]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [13]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

46×1 Matrix{Float64}:
 0.956628239318962
 0.9680924939488736
 1.0274913820485194
 1.0270310335343171
 0.8996549871193242
 0.9363876746151771
 0.9951109014647526
 1.1205601984214608
 0.9388725781539099
 0.90915395436343
 0.8774788580721995
 0.9011511111058725
 0.9948572292475704
 ⋮
 2.195783234015177
 3.040902937646202
 2.631343216737511
 3.1687096095329315
 2.726291608202368
 2.8590000046427835
 2.7946995704356583
 2.628917580837503
 2.948616224528425
 2.9953467657101065
 3.0742511111785356
 2.8728129977638677

In [14]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

0.9347826086956522

### 🟣 Method 2: Ridge
We will use the same function but set alpha to zero.

In [15]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

0.9347826086956522

### 🟣 Method 3: Elastic Net
We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

In [16]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

0.9347826086956522

### 🟣 Method 4: Decision Trees
We will use the package `DecisionTree`

In [17]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [18]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

0.8913043478260869

### 🟣 Method 5: Random Forests
The `RandomForestClassifier` is available through the `DecisionTree` package as well.

In [19]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 5.6
Avg Depth:  4.2

In [20]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

0.9130434782608695

### 🟣 Method 6: Using a Nearest Neighbor method
We will use the `NearestNeighbors` package here.

In [21]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

KDTree{StaticArrays.SVector{4, Float64}, Euclidean, Float64}
  Number of points: 104
  Dimensions: 4
  Metric: Euclidean(0.0)
  Reordered: true

In [22]:
queries = X[testids,:]

46×4 Matrix{Float64}:
 4.9  3.0  1.4  0.2
 5.0  3.4  1.5  0.2
 4.4  2.9  1.4  0.2
 4.8  3.4  1.6  0.2
 4.3  3.0  1.1  0.1
 5.4  3.9  1.3  0.4
 5.1  3.8  1.5  0.3
 4.8  3.4  1.9  0.2
 5.2  3.5  1.5  0.2
 5.2  3.4  1.4  0.2
 5.0  3.2  1.2  0.2
 4.9  3.6  1.4  0.1
 4.4  3.0  1.3  0.2
 ⋮              
 6.2  2.9  4.3  1.3
 6.5  3.0  5.8  2.2
 4.9  2.5  4.5  1.7
 7.2  3.6  6.1  2.5
 6.5  3.2  5.1  2.0
 6.8  3.0  5.5  2.1
 5.6  2.8  4.9  2.0
 6.1  3.0  4.9  1.8
 6.4  2.8  5.6  2.1
 6.4  2.8  5.6  2.2
 6.8  3.2  5.9  2.3
 6.7  3.0  5.2  2.3

In [23]:
idxs, dists = knn(kdtree, queries', 5, true)

([[25, 32, 9, 7, 18], [27, 1, 12, 19, 4], [3, 33, 32, 9, 2], [20, 19, 21, 6, 27], [33, 2, 3, 9, 6], [8, 34, 24, 5, 15], [15, 34, 4, 12, 1], [20, 19, 21, 17, 27], [1, 27, 12, 34, 4], [1, 27, 12, 28, 4]  …  [60, 59, 44, 37, 74], [81, 100, 68, 92, 84], [102, 76, 94, 77, 98], [96, 97, 81, 84, 77], [99, 67, 74, 104, 75], [95, 86, 104, 83, 90], [69, 73, 77, 94, 102], [69, 73, 97, 77, 102], [81, 84, 100, 97, 68], [98, 102, 96, 76, 97]], [[0.14142135623730964, 0.14142135623730986, 0.1414213562373099, 0.1732050807568878, 0.22360679774997896], [0.09999999999999964, 0.17320508075688762, 0.1999999999999999, 0.22360679774997902, 0.22360679774997916], [0.2999999999999997, 0.3605551275463988, 0.42426406871192807, 0.42426406871192807, 0.4358898943540674], [0.22360679774997858, 0.2828427124746191, 0.2999999999999998, 0.3000000000000002, 0.31622776601683783], [0.4795831523312718, 0.5000000000000003, 0.519615242270663, 0.58309518948453, 0.6164414002968974], [0.3464101615137753, 0.36055512754639896, 0.387

In [29]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(string(argmax(possible_labels[i])))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

0.9347826086956522

### 🟣 Method 7: Support Vector Machines
We will use the `LIBSVM` package here.

In [30]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

104-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [31]:
model = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Int64, Float64}(35, Int32[4, 15, 16], [1, 1, 1, 1, 2, 2, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [5.7 5.1 … 6.5 5.9; 4.4 3.3 … 3.0 3.0; 1.5 1.7 … 5.2 5.1; 0.4 0.5 … 2.0 1.8], Int32[11, 17, 29, 31, 35, 36, 37, 38, 39, 40  …  86, 87, 89, 90, 95, 98, 99, 101, 102, 104], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 4.5), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 4.9), LIBSVM.SVMNode(1, 5.0)  …  LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 7.2), LIBSVM.SVMNode(1, 7.9), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 6.9), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.5), LIBSVM.SVMNode(1, 5.9)]), 0.0, [0.37125893964830514 0.9086334573221102; 0.192673182555654 0.0; … ; -0.0 -0.72764858131

In [32]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

0.9347826086956522

Putting all the results together:

In [33]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

7×2 Matrix{Any}:
 "lasso"  0.934783
 "ridge"  0.934783
 "EN"     0.934783
 "DT"     0.891304
 "RF"     0.913043
 "kNN"    0.934783
 "SVM"    0.934783

# Finally...
After finishing this notebook, you should be able to:
- [ ] split your data into training and testing data to test the effectiveness of a certain method
- [ ] apply a simple accuracy function to test the effectiveness of a certain method
- [ ] run multiple classification algorithms:
    - [ ] LASSO
    - [ ] Ridge
    - [ ] ElasticNet
    - [ ] Decision Tree
    - [ ] Random Forest
    - [ ] Nearest Neighbors
    - [ ] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |1.0|
| ridge  |1.0|
| EN     |1.0|
| DT     |0.960784|
| RF     |0.980392|
| kNN    |1.0|
| SVM    |1.0|